In [1]:
from rdflib import Graph, URIRef, Literal, util
from rdflib.namespace import RDF, RDFS, XSD, OWL
import warnings
from pprint import pprint
import json

In [2]:
# base_namespaces_dict = {
#     "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
#     "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
#     "xsd": "http://www.w3.org/2001/XMLSchema#",
#     "owl": "http://www.w3.org/2002/07/owl#"
# }

# base_namespaces_list = list(base_namespaces_dict.values())
#print(base_namespaces_list)


In [3]:
# graph_base = Graph()
# graph_base.parse(str(RDF))
# graph_base.parse(str(RDFS))
# graph_base.parse(str(OWL))

# for s, p, o in graph_base:
#    print(s, p, o)

In [4]:
class UnexpectedRangeDatatype(UserWarning):
     pass

In [5]:
class MultipleRangeDatatypes(UserWarning):
     pass

In [6]:
g = Graph()
g.parse("http://www.w3.org/1999/02/22-rdf-syntax-ns#", format='xml')
g.parse("http://www.w3.org/2000/01/rdf-schema#", format='xml')
g.parse("http://www.w3.org/2002/07/owl#", format='xml')

<Graph identifier=N86e274e6c5a2424da52ca12d4bbb7e2b (<class 'rdflib.graph.Graph'>)>

In [7]:
# Get all subjects of type property, i.e. the properties declared for the business domain:
props = list(g.subjects(RDF.type, RDF.Property))
props.sort()
for p in props:
    print(p)

http://www.w3.org/1999/02/22-rdf-syntax-ns#direction
http://www.w3.org/1999/02/22-rdf-syntax-ns#first
http://www.w3.org/1999/02/22-rdf-syntax-ns#language
http://www.w3.org/1999/02/22-rdf-syntax-ns#object
http://www.w3.org/1999/02/22-rdf-syntax-ns#predicate
http://www.w3.org/1999/02/22-rdf-syntax-ns#rest
http://www.w3.org/1999/02/22-rdf-syntax-ns#subject
http://www.w3.org/1999/02/22-rdf-syntax-ns#type
http://www.w3.org/1999/02/22-rdf-syntax-ns#value
http://www.w3.org/2000/01/rdf-schema#comment
http://www.w3.org/2000/01/rdf-schema#domain
http://www.w3.org/2000/01/rdf-schema#isDefinedBy
http://www.w3.org/2000/01/rdf-schema#label
http://www.w3.org/2000/01/rdf-schema#member
http://www.w3.org/2000/01/rdf-schema#range
http://www.w3.org/2000/01/rdf-schema#seeAlso
http://www.w3.org/2000/01/rdf-schema#subClassOf
http://www.w3.org/2000/01/rdf-schema#subPropertyOf
http://www.w3.org/2002/07/owl#allValuesFrom
http://www.w3.org/2002/07/owl#annotatedProperty
http://www.w3.org/2002/07/owl#annotatedSour

In [8]:
# Classify business domain properties between attributes and relationships:
if len(props) > 0:
    props_dict = {}
    for prop in props:
        objects = list(g.objects(None, prop))   
        ###TODO:refactor above into a function which aggregates all objects per property into a set. For loop belowis iterating over a same object multiple times.
        num_objs = len(objects)
        prop_dict = {'number_of_triples': num_objs}
        if num_objs > 0:
            num_lits = 0
            num_uri = 0
            num_else = 0
            for obj in objects:
                if isinstance(obj, Literal):
                    num_lits += 1
                elif isinstance(obj, URIRef):
                    num_uri += 1
                else:
                    num_else += 1
            if num_else > 0:
                warning.warn("At least one object was neither a URI or a Literal.", UnexpectedRangeDatatype)
            if num_lits > 0 and num_uri >0:
                warnings.warn("At least one object was URI and another a Literal.", MultipleRangeDatatypes)
            if num_lits >= num_uri: 
                datatype_flag = "owl_DatatypeProperty" # This will assign any different datatype, multiple datatypes (lit > uri), and num_lits == 0 & num_uri == 0 as a graph attribute.
            else:
                datatype_flag = "owl_ObjectProperty"
            prop_dict['number_of_literals'] = num_lits
            prop_dict['number_of_uri'] = num_uri
            prop_dict['property_type'] = datatype_flag
            props_dict[prop] = prop_dict
        else:
            props_dict[prop] = prop_dict
        pprint(props_dict)
else:
    print("There are no properties declared for the business domain.")

                                                           'number_of_uri': 120,
                                                                         'property_type': 'owl_ObjectProperty'},
 rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#value'): {'number_of_triples': 0},
 rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#comment'): {'number_of_literals': 115,
                                                                      'number_of_triples': 115,
                                                                      'number_of_uri': 0,
                                                                      'property_type': 'owl_DatatypeProperty'},
 rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#domain'): {'number_of_literals': 0,
                                                                     'number_of_triples': 69,
                                                                     'number_of_uri': 69,
                                  

In [9]:
# Get property classification:
def get_property_classification(property_uri, property_dict):
    if property_dict[property_uri]['number_of_triples'] > 0:
            return(property_dict[property_uri]['property_type'])

In [10]:
# Define standard and business domain node attributes:
standard_attributes = [URIRef('http://www.w3.org/2000/01/rdf-schema#label'), 
                       URIRef('http://www.w3.org/2000/01/rdf-schema#comment'),
                       URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#language')]

domain_attributes = []

In [11]:
# Define standard and business domain relationships:
standard_relationships = [URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'),
                          URIRef('http://www.w3.org/2000/01/rdf-schema#subClassOf')]

domain_relationships = []

In [12]:
# Retrieve all business domain attributes and relationships:
for prop in props_dict:
    property_type = get_property_classification(prop, props_dict)
    if property_type == 'owl_DatatypeProperty':
        domain_attributes.append(prop)
    if property_type == 'owl_ObjectProperty':
        domain_relationships.append(prop)

print("Domain Attributes: {}".format(domain_attributes))
print("Domain Relationships: {}".format(domain_relationships))

Domain Attributes: [rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#comment'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#label')]
Domain Relationships: [rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#domain'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#isDefinedBy'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#range'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#seeAlso'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#subClassOf'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#subPropertyOf')]


In [13]:
# Aggregate all attributes and relationships:
all_attributes = standard_attributes + domain_attributes
all_relationships = standard_relationships + domain_relationships

pprint("All Attributes: {}".format(all_attributes))
pprint("All Relationships: {}".format(all_relationships))

('All Attributes: '
 "[rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#label'), "
 "rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#comment'), "
 "rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#language'), "
 "rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#comment'), "
 "rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#label')]")
('All Relationships: '
 "[rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), "
 "rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#subClassOf'), "
 "rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), "
 "rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#domain'), "
 "rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#isDefinedBy'), "
 "rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#range'), "
 "rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#seeAlso'), "
 "rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#subClassOf'), "
 "r

In [14]:
# Identify entities related by a property:
related_entities = set()
for dom_rel in domain_relationships:
    subs_objs = g.subject_objects(dom_rel)
    for s, o in subs_objs:
        related_entities.add(s)
        related_entities.add(o)
print(len(related_entities))

130


In [15]:
# Get all subjects of type class, i.e. the classes declared for the business domain:
declared_entities = list(g.subjects(RDF.type, RDFS.Class)) + list(g.subjects(RDF.type, OWL.Class)) 
declared_entities.sort()
for c in declared_entities:
    print(c)

http://www.w3.org/1999/02/22-rdf-syntax-ns#Alt
http://www.w3.org/1999/02/22-rdf-syntax-ns#Bag
http://www.w3.org/1999/02/22-rdf-syntax-ns#CompoundLiteral
http://www.w3.org/1999/02/22-rdf-syntax-ns#List
http://www.w3.org/1999/02/22-rdf-syntax-ns#Property
http://www.w3.org/1999/02/22-rdf-syntax-ns#Seq
http://www.w3.org/1999/02/22-rdf-syntax-ns#Statement
http://www.w3.org/2000/01/rdf-schema#Class
http://www.w3.org/2000/01/rdf-schema#Container
http://www.w3.org/2000/01/rdf-schema#ContainerMembershipProperty
http://www.w3.org/2000/01/rdf-schema#Datatype
http://www.w3.org/2000/01/rdf-schema#Literal
http://www.w3.org/2000/01/rdf-schema#Resource
http://www.w3.org/2002/07/owl#AllDifferent
http://www.w3.org/2002/07/owl#AllDisjointClasses
http://www.w3.org/2002/07/owl#AllDisjointProperties
http://www.w3.org/2002/07/owl#Annotation
http://www.w3.org/2002/07/owl#AnnotationProperty
http://www.w3.org/2002/07/owl#AsymmetricProperty
http://www.w3.org/2002/07/owl#Axiom
http://www.w3.org/2002/07/owl#Class


In [16]:
# Merge declared classes with related classes:
all_entities = related_entities
all_entities.update(declared_entities)
all_entities = list(all_entities)
all_entities.sort()
print(len(all_entities))
pprint(all_entities)

130
[rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#'),
 rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#Alt'),
 rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#Bag'),
 rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#CompoundLiteral'),
 rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#HTML'),
 rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#JSON'),
 rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#List'),
 rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#PlainLiteral'),
 rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#Property'),
 rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#Seq'),
 rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#Statement'),
 rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#XMLLiteral'),
 rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#direction'),
 rdflib.term.URIRef('http://

In [17]:
# Get all attributes for a given entity:
def get_node_attributes(entity, graph, attributes_list):
    node_attributes = {}
    for att in attributes_list:
        val = graph.objects(entity, att)
        val = util.first(val)
        node_attributes[att] = val
    return(node_attributes)

In [18]:
# Create node list:
node_list = {}
for ent in all_entities:
    entity_node = get_node_attributes(ent, g, all_attributes)
    node_list[ent] = entity_node
pprint(node_list)

{rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#'): {rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#language'): None,
                                                                     rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#comment'): None,
                                                                     rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#label'): None},
 rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#Alt'): {rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#language'): None,
                                                                        rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#comment'): rdflib.term.Literal('The class of containers of alternatives.'),
                                                                        rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#label'): rdflib.term.Literal('Alt')},
 rdflib.term.URIRef('http://www.w3.org/1

In [20]:
node_list_json = json.dumps(node_list)
pprint(node_list_json)

('{"http://www.w3.org/1999/02/22-rdf-syntax-ns#": '
 '{"http://www.w3.org/2000/01/rdf-schema#label": null, '
 '"http://www.w3.org/2000/01/rdf-schema#comment": null, '
 '"http://www.w3.org/1999/02/22-rdf-syntax-ns#language": null}, '
 '"http://www.w3.org/1999/02/22-rdf-syntax-ns#Alt": '
 '{"http://www.w3.org/2000/01/rdf-schema#label": "Alt", '
 '"http://www.w3.org/2000/01/rdf-schema#comment": "The class of containers of '
 'alternatives.", "http://www.w3.org/1999/02/22-rdf-syntax-ns#language": '
 'null}, "http://www.w3.org/1999/02/22-rdf-syntax-ns#Bag": '
 '{"http://www.w3.org/2000/01/rdf-schema#label": "Bag", '
 '"http://www.w3.org/2000/01/rdf-schema#comment": "The class of unordered '
 'containers.", "http://www.w3.org/1999/02/22-rdf-syntax-ns#language": null}, '
 '"http://www.w3.org/1999/02/22-rdf-syntax-ns#CompoundLiteral": '
 '{"http://www.w3.org/2000/01/rdf-schema#label": "CompoundLiteral", '
 '"http://www.w3.org/2000/01/rdf-schema#comment": "A class representing a '
 'compound li

In [21]:
entity_blacklist = [URIRef("http://www.w3.org/2000/01/rdf-schema#Class"),
                    URIRef("http://www.w3.org/1999/02/22-rdf-syntax-ns#Property")]

In [22]:
# Get all nodes for a given relation:
def get_relationship_triples(rel, graph, entity_blacklist):
    relationship_triples = []
    subs_objs = graph.subject_objects(rel)
    subs_objs = list(subs_objs)
    num_triples = len(subs_objs)
    if num_triples > 0:
        for s, o in subs_objs:
            if o not in entity_blacklist:
                relationship_triples.append(tuple((s, o)))

    relationship_triples_set = list(set([i for i in relationship_triples]))
    return(relationship_triples_set)

In [23]:
edge_list = {}
for rel in all_relationships:
    relationship_triples = get_relationship_triples(rel, g, entity_blacklist)
    edge_list[rel] = relationship_triples
pprint(edge_list)

{rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'): [(rdflib.term.URIRef('http://www.w3.org/2002/07/owl#incompatibleWith'),
                                                                          rdflib.term.URIRef('http://www.w3.org/2002/07/owl#OntologyProperty')),
                                                                         (rdflib.term.URIRef('http://www.w3.org/2002/07/owl#bottomObjectProperty'),
                                                                          rdflib.term.URIRef('http://www.w3.org/2002/07/owl#ObjectProperty')),
                                                                         (rdflib.term.URIRef('http://www.w3.org/2002/07/owl#topObjectProperty'),
                                                                          rdflib.term.URIRef('http://www.w3.org/2002/07/owl#ObjectProperty')),
                                                                         (rdflib.term.URIRef('http://www.w3.org/2002/07/owl#imports'

In [24]:
edge_list_json = json.dumps(edge_list)
pprint(edge_list_json)

('{"http://www.w3.org/1999/02/22-rdf-syntax-ns#type": '
 '[["http://www.w3.org/2002/07/owl#incompatibleWith", '
 '"http://www.w3.org/2002/07/owl#OntologyProperty"], '
 '["http://www.w3.org/2002/07/owl#bottomObjectProperty", '
 '"http://www.w3.org/2002/07/owl#ObjectProperty"], '
 '["http://www.w3.org/2002/07/owl#topObjectProperty", '
 '"http://www.w3.org/2002/07/owl#ObjectProperty"], '
 '["http://www.w3.org/2002/07/owl#imports", '
 '"http://www.w3.org/2002/07/owl#OntologyProperty"], '
 '["http://www.w3.org/2002/07/owl#priorVersion", '
 '"http://www.w3.org/2002/07/owl#OntologyProperty"], '
 '["http://www.w3.org/2002/07/owl#topDataProperty", '
 '"http://www.w3.org/2002/07/owl#DatatypeProperty"], '
 '["http://www.w3.org/2002/07/owl#versionInfo", '
 '"http://www.w3.org/2002/07/owl#AnnotationProperty"], '
 '["http://www.w3.org/2002/07/owl#backwardCompatibleWith", '
 '"http://www.w3.org/2002/07/owl#OntologyProperty"], '
 '["http://www.w3.org/1999/02/22-rdf-syntax-ns#XMLLiteral", '
 '"http://w